### Imports

In [30]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import calendar
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler





# Table 1


### Loading data

In [11]:
url = 'https://github.com/Sebasleen/Seminargroup/raw/Seminar/Data/anomalies.dta'

Anomalies = pd.read_stata(url)

# display unique values in the 'anomaly' column
print(Anomalies['anomaly'].unique())

# delete the global factors from the dataframe and create Anomalies US
column_name = 'anomaly'
values_to_dropUS = ['glbab', 'glcma', 'glhml', 'glqmj', 'glrmw', 'glsmb', 'glumd']
ElementsUS = Anomalies[column_name].isin(values_to_dropUS)
Anomalies_US = Anomalies[~ElementsUS]

# delete the US factors from dataframe and create Anomalies Global 
column_name = 'anomaly'
values_to_dropGF = ['ac', 'bab', 'cfp', 'cma', 'ep', 'hml', 'liq', 'ltrev', 'nsi', 'qmj', 'rmw', 'rvar',
                    'smb', 'strev', 'umd']
ElementsGF = Anomalies[column_name].isin(values_to_dropGF)
Anomalies_GF = Anomalies[~ElementsGF]

# print both anomalies US and anomalies global 
print(Anomalies_US)
print(Anomalies_GF)

['ac' 'bab' 'cfp' 'cma' 'ep' 'hml' 'liq' 'ltrev' 'nsi' 'qmj' 'rmw' 'rvar'
 'smb' 'strev' 'umd' 'glbab' 'glcma' 'glhml' 'glqmj' 'glrmw' 'glsmb'
 'glumd']
       year  month anomaly    ret   time  global
0      1963      7      ac  2.170   42.0     0.0
1      1963      8      ac -0.197   43.0     0.0
2      1963      9      ac  0.600   44.0     0.0
3      1963     10      ac  6.463   45.0     0.0
4      1963     11      ac -2.260   46.0     0.0
...     ...    ...     ...    ...    ...     ...
10111  2019      8     umd  7.600  715.0     0.0
10112  2019      9     umd -6.850  716.0     0.0
10113  2019     10     umd  0.240  717.0     0.0
10114  2019     11     umd -2.620  718.0     0.0
10115  2019     12     umd -2.130  719.0     0.0

[10116 rows x 6 columns]
       year  month anomaly       ret   time  global
10116  1987      2   glbab  2.236918  325.0     1.0
10117  1987      3   glbab  1.828450  326.0     1.0
10118  1987      4   glbab -5.521739  327.0     1.0
10119  1987      5   glba

### replicating table 1

### US factors

In [12]:
# calculate the mean and stand deviation of US factors
AnomaliesUS = Anomalies_US.groupby(['anomaly']).agg({'ret': ['mean', 'std', 'count']}).reset_index()
AnomaliesUS = Anomalies_US.pivot_table(index='anomaly', values='ret', aggfunc=['mean', 'std', 'count'])
AnomaliesUS.columns = ['Mean', 'SD', 'ret_number']
AnomaliesUS.reset_index(inplace=True)
AnomaliesUS.columns = ['anomaly', 'Mean', 'SD', 'ret_number']

# calculate additional statistics
AnomaliesUS['ret_semean'] = AnomaliesUS['SD'] / np.sqrt(AnomaliesUS['ret_number'])

# multiply by 12 to create annualized returns
AnomaliesUS['ret'] = AnomaliesUS['Mean'] * 12

# multiply by sqrt(12) to create annualized standard deviation
AnomaliesUS['sd'] = AnomaliesUS['SD'] * np.sqrt(12)

# calculate the t-stat by dividing the return by the standard error of the mean. Divide by 12 to annualize it
AnomaliesUS['tstat'] = AnomaliesUS['ret'] / AnomaliesUS['ret_semean'] /12

# format the table to correct decimals
AnomaliesUS[['Mean']] = AnomaliesUS[['ret']].apply(lambda x: x.map("{:.1f}%".format))
AnomaliesUS[['SD']] = AnomaliesUS[['sd']].apply(lambda x: x.map("{:.1f}%".format))
AnomaliesUS[['T-value']] = AnomaliesUS[['tstat']].apply(lambda x: x.map("{:.2f}".format))

print(AnomaliesUS[['anomaly','Mean','SD','T-value']])

   anomaly  Mean     SD T-value
0       ac  2.8%   6.6%    3.19
1      bab  9.8%  11.2%    6.55
2      cfp  3.4%   8.6%    2.94
3      cma  3.3%   6.9%    3.59
4       ep  3.5%   8.9%    2.95
5      hml  3.6%   9.7%    2.82
6      liq  4.4%  11.6%    2.77
7    ltrev  2.5%   8.7%    2.16
8      nsi  2.8%   8.2%    2.52
9      qmj  4.6%   7.7%    4.47
10     rmw  3.1%   7.5%    3.13
11    rvar  1.6%  17.3%    0.68
12     smb  2.7%  10.4%    1.97
13   strev  6.0%  10.6%    4.21
14     umd  7.8%  14.5%    4.02


### Global factors

In [13]:
# calculate the mean and stand deviation of Global factors
AnomaliesGF = Anomalies_GF.groupby(['anomaly']).agg({'ret': ['mean', 'std', 'count']}).reset_index()
AnomaliesGF = Anomalies_GF.pivot_table(index='anomaly', values='ret', aggfunc=['mean', 'std', 'count'])
AnomaliesGF.columns = ['Mean', 'SD', 'ret_number']
AnomaliesGF.reset_index(inplace=True)
AnomaliesGF.columns = ['anomaly', 'Mean', 'SD', 'ret_number']

# calculate additional statistics
AnomaliesGF['ret_semean'] = AnomaliesGF['SD'] / np.sqrt(AnomaliesGF['ret_number'])

# multiply by 12 to create annualized returns
AnomaliesGF['ret'] = AnomaliesGF['Mean'] * 12

# multiply by sqrt(12) to create annualized standard deviation
AnomaliesGF['sd'] = AnomaliesGF['SD'] * np.sqrt(12)

# calculate the t-stat by dividing the return by the standard error of the mean. Divide by 12 to annualize it
AnomaliesGF['tstat'] = AnomaliesGF['ret'] / AnomaliesGF['ret_semean'] /12

# format the table to correct decimals
AnomaliesGF[['Mean']] = AnomaliesGF[['ret']].apply(lambda x: x.map("{:.1f}%".format))
AnomaliesGF[['SD']] = AnomaliesGF[['sd']].apply(lambda x: x.map("{:.1f}%".format))
AnomaliesGF[['T-value']] = AnomaliesGF[['tstat']].apply(lambda x: x.map("{:.2f}".format))

print(AnomaliesGF[['anomaly','Mean','SD','T-value']])

  anomaly  Mean     SD T-value
0   glbab  9.6%   9.7%    5.70
1   glcma  1.9%   6.0%    1.74
2   glhml  4.0%   7.4%    2.92
3   glqmj  6.2%   6.8%    5.06
4   glrmw  4.3%   4.7%    4.91
5   glsmb  1.1%   7.1%    0.83
6   glumd  7.9%  12.1%    3.54


# Table 2 

### replicating table 2 (uses the same dataset as table 1)

In [15]:
# create an empty list to store results
results_list = []

# create a loop which iterates over each anomly in our dataset 
for anomaly in Anomalies['anomaly'].unique():
    subset = Anomalies[Anomalies['anomaly'] == anomaly]
    subset = subset.sort_values(by='time')

    # create a binary variable for positive returns in the past 12 months (the signal variable)
    subset['positive_return'] = subset['ret'].rolling(window=12, min_periods=12).mean().shift(1) > 0

    # drop the first 12 observations in the subset after the rolling window has been applied (so dropping N/A values)
    subset = subset.iloc[12:]

    # select our OLS model and fit our data
    y = subset['ret']
    X = sm.add_constant(subset['positive_return'].astype(int))
    model = sm.OLS(y, X)
    
    # select the correct covariance type (as used in the paper)
    results = model.fit(cov_type='cluster', cov_kwds={'groups': subset['time']})

    # append the results to our dictionary to create the results
    results_list.append({
        'anomaly': anomaly,
        'alpha': results.params['const'],
        'T-stat_alpha': results.tvalues['const'],
        'slope': results.params['positive_return'],
        'T-stat_slope': results.tvalues['positive_return'],
    })

results_table = pd.DataFrame(results_list)
print(results_table)


   Anomaly     Alpha  T-stat_Alpha     Slope  T-stat_Slope
0       ac  0.150195      1.184450  0.101410      0.649822
1      bab -0.221412     -0.632211  1.319041      3.534152
2      cfp  0.127745      0.781292  0.235454      1.157989
3      cma  0.120082      0.974474  0.244693      1.545819
4       ep  0.101357      0.616107  0.302075      1.458207
5      hml  0.038477      0.204762  0.410255      1.780679
6      liq  0.157215      0.741922  0.356063      1.291807
7    ltrev -0.252989     -1.663307  0.757680      3.850110
8      nsi  0.172982      1.324451  0.089249      0.486779
9      qmj  0.086832      0.650364  0.434757      2.507550
10     rmw  0.040360      0.222250  0.337185      1.673841
11    rvar -0.463569     -1.638345  1.061609      2.737366
12     smb -0.104191     -0.615583  0.583455      2.508982
13   strev  0.485098      1.427336  0.013888      0.038600
14     umd  0.716042      2.697340 -0.094969     -0.288098
15   glbab  0.190820      0.577502  0.837610      2.3039

# Table 3

### Loading dataset

In [5]:
# loading dataframes 

url = 'https://github.com/Sebasleen/Seminargroup/raw/Seminar/Data/managed_portfolios_anom_d_55.csv'

r_daily = pd.read_csv(url)

# drop all momentum factors or factors that are constructed based on momentum (including market return variables)

factor_drop_list = ['r_mom', 'r_indmom', 'r_valmom', 'r_valmomprof', 'r_mom12', 'r_momrev', 'r_indmomrev', 'r_exchsw', 'rme', 're_ew']

r_daily.drop(columns=factor_drop_list, inplace=True)

# set date to datetime format and set the date to the index 

r_daily['date'] = pd.to_datetime(r_daily['date'])
r_daily.set_index('date', inplace=True)

# following the procedure in the paper, if there are observations missing we set them to 0. (footnote 16)

r_daily.fillna(0, inplace=True)

# create a list of factors for later analysis purposes 

factors = [col for col in r_daily.columns if col.startswith('r_')]

# create a monthly return dataframe for later analysis purposes (by summing the daily returns)

r_monthly = r_daily.resample('ME').sum()
r_monthly.index = r_monthly.index.strftime('%Y-%m')


### perform the PCA analysis

In [31]:
# initialize pca model 

pca = PCA(n_components=len(factors))

scaler = StandardScaler()

# select our start date 

start_date = pd.to_datetime("1963-07-01")

# create an empty dataframe to store the average return for each PC from t until t-11. We need this to create the momentum signal for our strategy

pc_avg_df = pd.DataFrame()

# create an empty list for the pc return dataframes. These will be concated in a later stage to one large dataframe

pc_return_dfs = []

# create our loop set up, this is actually an expanding PCA analysis. In each iteration a new month is added to the dataset and the return is computed. 

for year in range(1973, 2020):
    # the sample of the paper starts from July 1973, but we use January to June 1973 to calculate the returns in order to obtain stable means for later demeaning purposes 
    for mo in range(1,13):
        # first we have to find the last month of the day. For this we use the calender function with inputs from the loop variables
        last_day = calendar.monthrange(year, mo)[1]

        # we select our new end_date variable for which the PCA analysis is done, also with inputs from our loop and the last_day variable
        end_date = pd.to_datetime(f'{year}-{mo}-{last_day}')

        t_dt = pd.to_datetime(f'{year}-{mo}')
        t = t_dt.strftime('%Y-%m')

        # we select the datarange from our dataset (July 1963 = start_date until our defined end_date) and we fit the model
        pca_data = r_daily.loc[start_date:end_date]
        scaled_data = scaler.fit_transform(pca_data)
        pca.fit(scaled_data)

        # we extract the principal components. These principal components are put in a new dataframe for later analysis. 

        principal_components = pca.components_
        components_df = pd.DataFrame(data=principal_components.T, index=factors, columns=[f"PC{i+1}" for i in range(len(factors))])

        # calculating return for month t+1. If mo = 12, then year will increment with 1. 

        t_plus_1_year = year + 1 if mo == 12 else year
        t_plus_1_month = (mo % 12) + 1

        # creating a datetime variable for the month t+1 and storing this in our pc_return_data variable

        t_plus_1_dt =pd.to_datetime(f'{t_plus_1_year}-{t_plus_1_month}')
        t_plus_1 = t_plus_1_dt.strftime('%Y-%m')

        pc_return_data = {'date': t_plus_1}


        # in this loop we calculate the monthly factor returns (f) using the principal components and returns

        for f in range(len(factors)):
            # select our factor and extract its principal component from principal_df and its return from r_daily for all observations in month mo 
            pc = components_df.iloc[:, f]
            r_month = r_monthly.loc[t]
            # multiply the principal components with the returns and sum them up to get PC factor return for month mo 
            pc_return = (pc*r_month).sum()

            # place this in our dictionary for later transposing to dataframe

            pc_return_data[components_df.columns[f]] = pc_return

            r_pc_month_n_list = []
            
            # in this loop we calculate the average return of the eigenvector at time t, for the period t until t-11. We store these results in a dataframe for later use.

            for n in range(0, 12):
                # calculate the datetime for t - n
                t_minus_n_dt = t_dt - pd.DateOffset(months=n)

                # transpose it to our YYYY-MM format
                t_minus_n = t_minus_n_dt.strftime('%Y-%m')

                # select the return corresponding to our month t-n
                r_month_n = r_monthly.shift(n).loc[t_minus_n]

                # calculate the dot product for month t-n
                pc_return_n = (pc*r_month_n).sum()
                
                # append this to our list to calculate the mean 
                r_pc_month_n_list.append(pc_return_n)
            
            # calculate the mean and append it to our average return dataframe
            r_pc_month_mean = (np.mean(r_pc_month_n_list))
            pc_avg_df.loc[t, f'PC{f+1}'] = r_pc_month_mean

        # append the PC returns to our PC_return dataframe for later analysis 
        pc_return_df = pd.DataFrame.from_dict(pc_return_data, orient='index').T
        pc_return_df.set_index('date', inplace=True)
        pc_return_dfs.append(pc_return_df)

r_pc = pd.concat(pc_return_dfs)
print(r_pc)

              PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
date                                                                            
1973-02 -0.163527  -0.10435 -0.004879  0.001906  0.080599   0.11734  0.004165   
1973-03 -0.254274  -0.04289 -0.002538 -0.097974 -0.033213 -0.057495   0.00577   
1973-04 -0.166303  0.053257  0.051769 -0.043656  0.022396  0.051989    0.0334   
1973-05 -0.403385  0.076711   0.09642  -0.07831    0.0444  0.073014  0.079326   
1973-06 -0.100178 -0.117113 -0.034034 -0.069168  0.000694  0.010336  0.061616   
...           ...       ...       ...       ...       ...       ...       ...   
2019-09    0.0618  -0.26667   -0.0452  0.144064  0.059383 -0.041025  -0.05225   
2019-10 -0.418177  0.059967  0.232239 -0.056775 -0.086553 -0.024549  0.072874   
2019-11  0.035635 -0.109379  0.016161 -0.062541    0.0222  0.060263  0.078533   
2019-12   0.20419  0.041358 -0.038417   0.01059 -0.027231  0.046026   0.01053   
2020-01 -0.072692  0.050692 

### demeaning and leveraging our PC returns

In [37]:
# define our start date and create an empty list for our leveraged dataframes
start_date_dt = pd.to_datetime("1963-07-01")
start_date = start_date_dt.strftime('%Y-%m')
lev_dfs = []

for year in range(1973, 2020):
    # as we lost one month in calculating the t+1 PC return, we start the loop from february 1973. We try to use the full year of 1973 in order to obtain stable demeaned results. Hence we will cut of our sample later from July 1973 in order to match the dataset of the original paper. 
    for mo in range(2,13) if year == 1973 else range(1, 13):
        
        # first we set our t variable to the current year and month from our loop
        t_dt = pd.to_datetime(f'{year}-{mo}')
        t = t_dt.strftime('%Y-%m')
        
        # we also create a t_minus_one variable, because we have to calculate the variance up to month t (so excluding month t)
        t_minus_one_dt = t_dt - pd.DateOffset(months=1)
        t_minus_one = t_minus_one_dt.strftime('%Y-%m')

        # calculate the variance of the individual factor returns up until month t-1 
        r_indiv_f_t = r_monthly.loc[start_date:t_minus_one]
        var_indiv_f_t = r_indiv_f_t.var(axis=0)
        avg_var_indiv_f_t = var_indiv_f_t.mean()

        # calculate the mean and variance of the PC factors up until month t 
        r_pc_t = r_pc.loc[:t]
        demeaned_r_pc_t = r_pc.loc[t].to_frame().T - r_pc_t.mean()

        # calculate the leverage factor and multiply this with the demeaned 

        leverage_t = np.divide(np.sqrt(avg_var_indiv_f_t), r_pc_t.std(axis=0), where=r_pc_t.std(axis=0)!=0)
        lev_r_pc_t = demeaned_r_pc_t * leverage_t
        lev_df = lev_r_pc_t.loc[t].to_frame().T
        lev_dfs.append(lev_df)

lev_r_pc = pd.concat(lev_dfs)
lev_r_pc.fillna(0, inplace=True)
lev_r_pc_clean = lev_r_pc.drop(lev_r_pc.index[:1])
print(lev_r_pc_clean)

              PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
1973-03 -0.026649  0.026649  0.026649 -0.026649 -0.026649 -0.026649  0.026649   
1973-04  0.020729  0.040113  0.043471  0.002199 -0.000572  0.006273  0.043448   
1973-05 -0.052914  0.036520  0.048040 -0.020672  0.012657  0.013721  0.052732   
1973-06  0.038003 -0.038651 -0.040384 -0.011589 -0.019663 -0.016462  0.028198   
1973-07  0.029864 -0.006144  0.017073 -0.039421 -0.018559  0.019224 -0.011339   
...           ...       ...       ...       ...       ...       ...       ...   
2019-08  0.015596 -0.013605 -0.002106  0.023654 -0.027606  0.000796  0.032523   
2019-09  0.018075 -0.107229 -0.023939  0.075979  0.038220 -0.035257 -0.040602   
2019-10 -0.083752  0.017626  0.127434 -0.043139 -0.056248 -0.022306  0.057079   
2019-11  0.012635 -0.047106  0.009422 -0.046467  0.014220  0.044328  0.061344   
2019-12  0.048361  0.010570 -0.020369 -0.003046 -0.017808  0.033079  0.008221   

              PC8       PC9

/var/folders/zx/2ff938wn7d11yl1k3djhsp880000gn/T/ipykernel_92321/2738485239.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lev_r_pc.fillna(0, inplace=True)


### constructing the momentum strategy

In [39]:
# create two boolean dataframes: one for positive average returns and one for negative average returns
positive_returns_PC = pc_avg_df > 0
negative_returns_PC = pc_avg_df < 0

# convert the boolean dataframes to integers and 0's, one for long positions and one for short positions
long_portfolio_PC = positive_returns_PC.astype(int)
short_portfolio_PC = negative_returns_PC.astype(int)

# create the 5 subsets of PCs
mom_1_10 = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10']
mom_11_20 = ['PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20']
mom_21_30 = ['PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27', 'PC28', 'PC29', 'PC30']
mom_31_40 = ['PC31', 'PC32', 'PC33', 'PC34', 'PC35', 'PC36', 'PC37', 'PC38', 'PC39', 'PC40']
mom_41_47 = ['PC41', 'PC42', 'PC43', 'PC44', 'PC45', 'PC46', 'PC47']

# create a list of the subsets for our loop
mom_list = [mom_1_10, mom_11_20, mom_21_30, mom_31_40, mom_41_47]

# create an empty dictionary 
r_mean_set_dict = {}

# create a loop where the dummy dataframe is multiplied with the leveraged PC return dataframe. We shift the portfolio indicator with one, as we need to calculate the returns of t+1. 
for i, mom in enumerate(mom_list):
    # create the strategy: the return of the long positions minus the return of the short positions (accounting for the fact that negative short returns need to become positive
    r_PC_set_mom = (long_portfolio_PC[mom] * lev_r_pc_clean[mom]) - (short_portfolio_PC[mom] * lev_r_pc_clean[mom])
    # we take the mean of the returns of the 10 PC subsets
    r_PC_set_mean = r_PC_set_mom.mean(axis=1)
    # we append it to our dictionary
    r_mean_set_dict[f'mom_set_{i + 1}'] = r_PC_set_mean

# create the dataframe with the series of returns for each subset of PCS
mom_strategy = pd.concat(r_mean_set_dict, axis=1)

mom_strategy.index = pd.to_datetime(mom_strategy.index)
mom_strategy.index = mom_strategy.index.strftime('%Y-%m')
mom_strategy.dropna(inplace=True)

print(mom_strategy.mean(axis=0))

mom_set_1    0.010090
mom_set_2    0.010919
mom_set_3    0.012050
mom_set_4    0.010571
mom_set_5    0.009817
dtype: float64


### Replicating table 3

### replicating panel A

In [ ]:
# select the full sample dataframe in the paper and create the two splitted periods 
mom_strategy_full = mom_strategy.loc['1973-07':'2019-12']
mom_strategy_1 = mom_strategy.loc['1973-07':'1996-09']
mom_strategy_2 = mom_strategy.loc['1996-09':]

print(f'the mean of every subset of PCs is:\n')
print(mom_strategy_full.mean(axis=0))

means = mom_strategy.mean(axis=0).tolist()
std = mom_strategy.std(axis=0).tolist()
N = mom_strategy.shape[0]

print(f'the t-statistic of every subset of PCS is:\n')
for m, s in zip(means, std):
    t_statistic = m / (s / (N**0.5))
    print(t_statistic)

print(f'the mean of every subset of PCs is (first half):\n')
print(mom_strategy_1.mean(axis=0))

means = mom_strategy_1.mean(axis=0).tolist()
std = mom_strategy_1.std(axis=0).tolist()
N = mom_strategy_1.shape[0]

print(f'the t-statistic of every subset of PCS is (first half):\n')
for m, s in zip(means, std):
    t_statistic = m / (s / (N**0.5))
    print(t_statistic)

print(f'the mean of every subset of PCs is (second half):\n')
print(mom_strategy_2.mean(axis=0))

means = mom_strategy_2.mean(axis=0).tolist()
std = mom_strategy_2.std(axis=0).tolist()
N = mom_strategy_2.shape[0]

print(f'the t-statistic of every subset of PCS is (second half):\n')
for m, s in zip(means, std):
    t_statistic = m / (s / (N**0.5))
    print(t_statistic)

### Replicating panel B and C

### Loading data and merging datasets

In [ ]:
url = 

ff = pd.read_stata("fffactors.dta")

ff.set_index('yyyymm', inplace=True)


ff.index = pd.to_datetime(ff.index, format='%Y%m')


ff.index = ff.index.strftime('%Y-%m')

ff5 = ff[['mktrf', 'smb', 'hml', 'rmw', 'cma']].loc['1973-08':'2019-12']

mom_strategy_ff5 = pd.concat([mom_strategy, ff5], axis=1)

mom_strategy_ff5['P1'] = 0
mom_strategy_ff5['P2'] = 0
mom_strategy_ff5.loc[mom_strategy_ff5.index <= '1996-09', 'P1'] = 1
mom_strategy_ff5.loc[mom_strategy_ff5.index >= '1996-09', 'P2'] = 1


In [44]:
# initialize pca model 

pca = PCA(n_components=len(factors))

scaler = StandardScaler()

# select our start date 

start_date = pd.to_datetime("1963-07-01")

# create an empty dataframe to store the average return for each PC from t until t-11. We need this to create the momentum signal for our strategy

pc_avg_df = pd.DataFrame()

# create an empty list for the pc return dataframes. These will be concated in a later stage to one large dataframe

pc_return_dfs = []

# create our loop set up, this is actually an expanding PCA analysis. In each iteration a new month is added to the dataset and the return is computed. 

for year in range(1973, 1974):
    # the sample of the paper starts from July 1973, but we use January to June 1973 to calculate the returns in order to obtain stable means for later demeaning purposes 
    for mo in range(1,13):
        # first we have to find the last month of the day. For this we use the calender function with inputs from the loop variables
        last_day = calendar.monthrange(year, mo)[1]

        # we select our new end_date variable for which the PCA analysis is done, also with inputs from our loop and the last_day variable
        end_date = pd.to_datetime(f'{year}-{mo}-{last_day}')

        t_dt = pd.to_datetime(f'{year}-{mo}')
        t = t_dt.strftime('%Y-%m')

        # we select the datarange from our dataset (July 1963 = start_date until our defined end_date) and we fit the model
        pca_data = r_daily.loc[start_date:end_date]
        scaled_data = scaler.fit_transform(pca_data)
        pca.fit(scaled_data)
        
        # we extract the principal components. These principal components are put in a new dataframe for later analysis. 

        principal_components = pca.components_
        components_df = pd.DataFrame(data=principal_components.T, index=factors, columns=[f"PC{i+1}" for i in range(len(factors))])

        # calculating return for month t+1. If mo = 12, then year will increment with 1. 

        t_plus_1_year = year + 1 if mo == 12 else year
        t_plus_1_month = (mo % 12) + 1

        # creating a datetime variable for the month t+1 and storing this in our pc_return_data variable

        t_plus_1_dt =pd.to_datetime(f'{t_plus_1_year}-{t_plus_1_month}')
        t_plus_1 = t_plus_1_dt.strftime('%Y-%m')

        pc_return_data = {'date': t_plus_1}


        # in this loop we calculate the monthly factor returns (f) using the principal components and returns

        for f in range(len(factors)):
            # select our factor and extract its principal component from principal_df and its return from r_daily for all observations in month mo 
            pc = components_df.iloc[:, f]
            r_month = r_monthly.loc[t]
            # multiply the principal components with the returns and sum them up to get PC factor return for month mo 
            pc_return = (pc*r_month).sum()

            # place this in our dictionary for later transposing to dataframe

            pc_return_data[components_df.columns[f]] = pc_return

            r_pc_month_n_list = []

            # in this loop we calculate the average return of the eigenvector at time t, for the period t until t-11. We store these results in a dataframe for later use.

            for n in range(0, 12):
                # calculate the datetime for t - n
                t_minus_n_dt = t_dt - pd.DateOffset(months=n)

                # transpose it to our YYYY-MM format
                t_minus_n = t_minus_n_dt.strftime('%Y-%m')

                # select the return corresponding to our month t-n
                r_month_n = r_monthly.shift(n).loc[t_minus_n]

                # calculate the dot product for month t-n
                pc_return_n = (pc*r_month_n).sum()

                # append this to our list to calculate the mean 
                r_pc_month_n_list.append(pc_return_n)

            # calculate the mean and append it to our average return dataframe
            r_pc_month_mean = (np.mean(r_pc_month_n_list))
            pc_avg_df.loc[t, f'PC{f+1}'] = r_pc_month_mean

        # append the PC returns to our PC_return dataframe for later analysis 
        pc_return_df = pd.DataFrame.from_dict(pc_return_data, orient='index').T
        pc_return_df.set_index('date', inplace=True)
        pc_return_dfs.append(pc_return_df)

r_pc = pd.concat(pc_return_dfs)
print(r_pc)

TypeError: '<' not supported between instances of 'str' and 'Timestamp'

In [46]:
pca = PCA(n_components=len(factors))
pca_data = r_daily
monthly_return = r_monthly
scaled_data = scaler.fit_transform(pca_data)
pca.fit(scaled_data)
pca_transformed_monthly = pca.transform(monthly_return)
print(pca_transformed_monthly)




[[ 0.01371338 -0.03463584 -0.00231046 ... -0.00397385 -0.00074066
   0.00052336]
 [ 0.03500007 -0.02204684  0.00457098 ...  0.00930063  0.00136526
  -0.00359697]
 [-0.05356279  0.04275117  0.0175099  ... -0.00051629  0.00281623
  -0.0041991 ]
 ...
 [ 0.03559436 -0.1092548   0.01666224 ... -0.00688367 -0.00059392
   0.000511  ]
 [ 0.20421029  0.04135745 -0.03841624 ...  0.00166796 -0.01164272
  -0.00676036]
 [-0.0726915   0.05069172  0.06717605 ...  0.00049199  0.00583927
   0.00102421]]


/Users/sjoerd/anaconda3/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but PCA was fitted without feature names
  warnings.warn(


In [48]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import calendar

# Assuming r_daily and r_monthly are your daily and monthly returns DataFrames
# Assuming factors is a list of your factor names

pca = PCA(n_components=len(factors))
scaler = StandardScaler()

start_date = pd.to_datetime("1963-07-01")
pc_avg_df = pd.DataFrame()
pc_return_dfs = []

# Loop setup for an expanding PCA analysis
for year in range(1973, 1974):
    for mo in range(1, 13):
        last_day = calendar.monthrange(year, mo)[1]
        end_date = pd.to_datetime(f'{year}-{mo}-{last_day}')
        t_dt = pd.to_datetime(f'{year}-{mo}')
        t = t_dt.strftime('%Y-%m')

        # Fit the PCA model on the scaled daily data up to end_date
        pca_data = r_daily.loc[start_date:end_date, factors]  # Ensure pca_data only contains factors
        scaled_data = scaler.fit_transform(pca_data)
        pca.fit(scaled_data)

        # For calculating return for month t+1
        t_plus_1_year = year + 1 if mo == 12 else year
        t_plus_1_month = 1 if mo == 12 else mo + 1

        t_plus_1_dt = pd.to_datetime(f'{t_plus_1_year}-{t_plus_1_month}')
        t_plus_1 = t_plus_1_dt.strftime('%Y-%m')

        # Prepare the monthly data for month t+1 for transformation
        
        r_month_t = r_monthly.loc[t]  # Ensure it's correctly indexed or adjusted
        scaled_month_t = scaler.transform([r_month_t])  # Scale the data
        pca_transformed_month_t = pca.transform(scaled_month_t)  # Transform the data

            # Calculate the returns for the PC factors for month t+1
        pc_return_data = {'date': t_plus_1}
        for i, pc_return in enumerate(pca_transformed_month_t[0]):
            pc_return_data[f'PC{i+1}'] = pc_return

            # Append the PC returns to our dataframe for later analysis
        pc_return_df = pd.DataFrame(pc_return_data, index=[0])
        pc_return_df.set_index('date', inplace=True)
        pc_return_dfs.append(pc_return_df)

r_pc = pd.concat(pc_return_dfs)
print(r_pc)

/Users/sjoerd/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/sjoerd/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/sjoerd/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/sjoerd/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/sjoerd/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/sjoerd/anaconda3/lib/python3.11/site-packag

               PC1        PC2        PC3        PC4        PC5        PC6  \
date                                                                        
1973-02 -23.819597 -18.110176  -3.918106  -1.951140  30.803665  54.373116   
1973-03 -40.839295  -4.651643   1.345406 -16.916963 -18.491059 -16.981794   
1973-04 -27.441461  10.466830  10.094875  -6.102751   6.482101  15.221173   
1973-05 -66.470072  16.418109  17.359363 -24.754827  24.557257  17.761599   
1973-06 -14.907274 -18.946135  -8.718853 -19.180315   2.640592  11.187002   
1973-07 -17.981296  -5.834703   4.174736 -25.901462   3.282226  18.966506   
1973-08  83.773174  -6.100417   6.110009  62.934446  -9.763654  -5.781026   
1973-09  -9.175734   0.831915   1.486356   4.791130  -2.737895  -0.185929   
1973-10  -0.642124  27.627757  17.384921  21.869598  -6.091616 -24.626278   
1973-11 -39.805662  10.674758   7.325617   4.736953   7.364143   2.182930   
1973-12 -61.032372 -10.701561  20.005871 -59.559890  48.503801  63.223974   

/Users/sjoerd/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/sjoerd/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
